In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut

showInfo = 0
p = ut.leer_csv("data/ARIA/4.PatientDemographics/_Patient.csv", showInfo).drop(columns = ["SSN","WorkPhone","HomePhone",
                      "Citizenship","MobilePhone","CreationDate","Language","PatientId2","MobilePhoneProviderSer","PatientId",
                       "FileDataSiteID","PatientUID"])

pa = ut.leer_csv("data/ARIA/4.PatientDemographics/_PatientAddress.csv", showInfo).drop(columns = ["PrimaryFlag"])

a  = ut.leer_csv("data/ARIA/4.PatientDemographics/_Address.csv", showInfo).drop(columns = ["County","AddressLine1",
                     "AddressLine2","PhoneNumber1","PhoneNumber2","EMailAddress","OriginationDate",
                      "TerminationDate","PostalCode","AddressType","StateOrProvince"])

# pp = ut.leer_csv("data/ARIA/4.PatientDemographics/_PatientParticular.csv", showInfo).drop(columns = ["MaritalStatus",
#          "RetireDate","DeathDate","PassportNumber","UserDefAttrib01","UniversalPatientId","RetireReason","RetireNote"])

In [19]:
# ut.info_tabla(a)

In [20]:
join1 = pa.merge(a, on="AddressSer", how="left").groupby("PatientSer", as_index=False).first()
join2 = p.merge(join1, on="PatientSer", how='left').drop(columns="AddressSer")
# join3 = join2.merge(pp, on="PatientSer", how="left")
join2.head()

,Sex,PatientSer,DateOfBirth,FirstName,Country,CityOrTownship
0,Other,4,NaN,Truebeam,None,None
1,Unknown,7,4/29/2008 12:00:00 AM,NaN,None,None
2,Unknown,8,4/29/2008 12:00:00 AM,NaN,None,None
3,Unknown,9,5/19/2008 12:00:00 AM,NaN,None,None
4,Unknown,11,4/29/2008 12:00:00 AM,NaN,None,None


In [22]:
join2["DateOfBirth"] = pd.DatetimeIndex( pd.to_datetime(join2["DateOfBirth"])).year
join2.head()

,Sex,PatientSer,DateOfBirth,FirstName,Country,CityOrTownship
0,Other,4,NaN,Truebeam,None,None
1,Unknown,7,2008.0,NaN,None,None
2,Unknown,8,2008.0,NaN,None,None
3,Unknown,9,2008.0,NaN,None,None
4,Unknown,11,2008.0,NaN,None,None


# Separar pacientes reales y "de prueba"

In [23]:
l2 = list(pd.read_csv("data/nombres_hombre.csv", skiprows=6, delimiter=";").Nombre.values)
l3 = list(pd.read_csv("data/nombres_mujer.csv" , skiprows=6, delimiter=";").Nombre.values)

import unidecode
l4 = []
for i in l2+l3:
    if isinstance(i, str): l4.append(unidecode.unidecode(i))

In [25]:
nombres = join2.FirstName.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') \
            .str.upper().str.split().str[0]

In [26]:
patient_real = join2[nombres.isin(l4)]
patient_test = join2[~nombres.isin(l4)]

In [ ]:
# patient_real
print(join2.shape)

print(patient_real.shape)
print(patient_test.shape)
# patient_real.head()

In [9]:
# patient_real[patient_real.DateOfBirth.isna()][0:60]
# patient_real[patient_real.PatientId2.isna()][0:60]
# patient_real.isna().sum()

# Guardar tablas

In [28]:
patient_real.to_csv("data/ARIA_merged&selected/Patient_real.csv", index=False)
patient_test.to_csv("data/ARIA_merged&selected/Patient_test.csv", index=False)